# Sentiment Analysis

In [3]:
!pip install pandas
!pip install numpy
!pip install sklearn

  Using cached pandas-1.4.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached numpy-1.23.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached pytz-2022.2.1-py2.py3-none-any.whl (500 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.1.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
  Using cached scipy-1.9.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.4 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import unicodedata
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB # cuando es texto se usa la multinomial
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

In [5]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


La base de datos cuenta con 891 observaciones y con 3 variables, de las cuales dos están clasificadas como objetos y una como entera.

In [7]:
df.sample(10)

,package_name,review,polarity
477,com.Slack,it sucks! i am always shown offline and unab...,0
506,com.Slack,brilliant! the best application i've found so...,1
229,com.supercell.clashofclans,sucks the new update especially the multiplay...,0
88,com.linkedin.android,"better user interface, worse quality the user...",0
214,com.supercell.clashofclans,once a great game i was a hardcore fan of thi...,0
610,com.evernote,good but ui ease of use still declining i've ...,1
466,com.whatsapp,no stickers!!!! why is there no stickers?it i...,0
873,com.rovio.angrybirds,such an awesome game love it a really fun gam...,1
390,com.google.android.talk,mic on speaker is not working with oneplusx w...,0
485,com.Slack,best app i can't believe that it is free! the...,1


Se observa que las dos primeras variables deberían ser string mientras que la última debería ser categórica, por lo que se procede a hacer las transformaciones.

In [8]:
df[df.select_dtypes('object').columns]=df[df.select_dtypes('object').columns].astype('string')
df['polarity']=df['polarity'].astype('category')

A continuación se eliminan algunos caracteres desconocidos que se detectaron en el texto.

In [9]:
df['review']=df['review'].str.strip() # elimina espacios al comienzo y al final de la oracion
df['review']=df['review'].str.lower() # lleva todo a minuscula

In [10]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [11]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True) # elimina caracteres repetidos mas de dos veces


In [12]:
df.sample(10)

,package_name,review,polarity
179,com.imangi.templerun2,temple run -pretty good the game is fun.but i ...,0
881,com.rovio.angrybirds,game ruined because of ads. i felt like re-dow...,0
248,com.android.chrome,image search on verizon ellipsis 10 tablet doe...,0
508,com.Slack,excellent app. clearly a passion project. grea...,1
520,com.dropbox.android,its not safe it has a link to gallery and file...,0
543,com.dropbox.android,very useful thank you android for introducing ...,1
199,com.supercell.clashofclans,fix or i quit don't download this game the new...,0
817,com.hamropatro,excellent awesome app very useful thanks forum...,1
165,com.imangi.templerun2,awesome its nice but not compatible with my ha...,0
1,com.facebook.katana,messenger issues ever since the last update in...,0


In [13]:
df.iloc[675,]

package_name                                    com.hamrokeyboard
review          loved it rzrl app rrlrrl rrsrzrlrrl r rrsrzrlr...
polarity                                                        1
Name: 675, dtype: object

In [14]:
X=df['review']
y=df['polarity']

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2007,stratify=y)

In [16]:
vec=CountVectorizer(stop_words='english')
X_train=vec.fit_transform(X_train).toarray()
X_test=vec.transform(X_test).toarray()

In [17]:
X_train.shape

(668, 3142)

In [18]:
nb=MultinomialNB()
nb.fit(X_train,y_train)
print('R score', nb.score(X_train,y_train))

R score 0.9580838323353293


In [19]:
y_predict=nb.predict(X_test)

In [20]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87       146
           1       0.78      0.65      0.71        77

    accuracy                           0.82       223
   macro avg       0.81      0.78      0.79       223
weighted avg       0.81      0.82      0.81       223



Se observa que el ajuste en la muestra de prueba es bastante bueno para predecir cualquiera de las categorias, hasta para la minoritaria el recall es mayor al 60%.

In [21]:
import pickle
filename = '/workspace/Naive-Bayes/models/finalized_model.sav'
pickle.dump(nb, open(filename, 'wb'))